In [2]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [4]:
with open('new_york_data.json') as json_data:
    new_york_data = json.load(json_data)

In [5]:
neighborhoods_data = new_york_data['features']

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of New York City: {}, {}.'.format(latitude, longitude))

Coordinates of New York City: 40.7127281, -74.0060152.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_newyork)  
    
map_newyork

In [11]:
queens_data = neighborhoods[neighborhoods["Borough"] == "Queens"].reset_index(drop = True)
print(queens_data.shape)
queens_data.head()

(81, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [13]:

address = 'Queens, NY'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_queens = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng,label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = '{} {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
                        [lat,lng],
                        radius = 5,
                        popup = label,
                        color = 'red',
                        fill = True,
                        fill_color = '#FF6347',
                        fill_opacity = 0.7).add_to(map_queens)
map_queens

In [14]:
CLIENT_ID = 'MAOYWB55HYDW3CKNLYOE2W3ZUEAX4JOKNZEBVMDIOPGTLYZF'
 # your Foursquare ID
CLIENT_SECRET = 'C5HZZIHALRRFOZS4X1II44L3SYROOKZYLYG4JYESH1EZYOLB'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MAOYWB55HYDW3CKNLYOE2W3ZUEAX4JOKNZEBVMDIOPGTLYZF
CLIENT_SECRET:C5HZZIHALRRFOZS4X1II44L3SYROOKZYLYG4JYESH1EZYOLB


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [35]:
print(queens_venues.shape)
queens_venues.head()

(1538, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym


In [36]:
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [37]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

#moving neighborhood to the front of the data frame
cols = queens_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
queens_onehot = queens_onehot.reindex(columns= cols)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool Hall,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Rental Service,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [38]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool Hall,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Rental Service,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arverne,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.055556,0.0,0.055556,0.0,0.0,0.055556,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0

In [39]:
len(queens_grouped[queens_grouped["Bar"]>0])

17

In [40]:
bar_df = queens_grouped[["Neighborhood","Bar"]]
print(bar_df.shape)
bar_df.head()

(81, 2)


,Neighborhood,Bar
0,Arverne,0.000000
1,Astoria,0.033333
2,Astoria Heights,0.000000
3,Auburndale,0.000000
4,Bay Terrace,0.000000


In [41]:
k_clusters = 8
queens_cluster = bar_df.drop(["Neighborhood"],1)

kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(queens_cluster)
kmeans.labels_[0:]

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 4, 0, 4, 0, 0, 2, 0, 2,
       4, 0, 0, 0, 0, 7, 0, 0, 0, 0, 5, 0, 0, 0, 1])

In [42]:
bar_cpy = bar_df.copy()

# add clustering labels
bar_cpy["Cluster Labels"] = kmeans.labels_
bar_cpy.rename(columns = {'Neighborhood':'Neighborhood'},inplace = True)
bar_cpy.head()

,Neighborhood,Bar,Cluster Labels
0,Arverne,0.000000,0
1,Astoria,0.033333,2
2,Astoria Heights,0.000000,0
3,Auburndale,0.000000,0
4,Bay Terrace,0.000000,0


In [43]:
bar_cpy = bar_cpy.join(queens_venues.set_index("Neighborhood"), on = "Neighborhood")
print(bar_cpy.shape)
bar_cpy.head()

(1538, 9)


,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arverne,0.0,0,40.589144,-73.791992,Rockaway Beach - 60th Street,40.589328,-73.789144,Beach
0,Arverne,0.0,0,40.589144,-73.791992,Far Rockaway Beach,40.588139,-73.795659,Surf Spot
0,Arverne,0.0,0,40.589144,-73.791992,Rockaway Beach - 69th Street,40.587350,-73.797250,Surf Spot
0,Arverne,0.0,0,40.589144,-73.791992,Locals Surf School,40.587016,-73.796337,Surf Spot
0,Arverne,0.0,0,40.589144,-73.791992,Locals Collective NYC,40.590395,-73.797677,Coffee Shop


In [44]:
bar_cpy.sort_values(["Cluster Labels"],inplace = True)
bar_cpy.head()

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arverne,0.0,0,40.589144,-73.791992,Rockaway Beach - 60th Street,40.589328,-73.789144,Beach
40,Jamaica Hills,0.0,0,40.711460,-73.796465,"MTA NYCT Q1, Q2, Q3, Q36, Q76, Q77 NICEBus N1,...",40.709230,-73.797299,Bus Station
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Taco Bell,40.715374,-73.793872,Fast Food Restaurant
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Pollo Popular,40.710027,-73.793734,Restaurant
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Jamaica Colosseum Mall,40.707243,-73.796147,Shopping Mall


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bar_cpy['Neighborhood Latitude'], bar_cpy['Neighborhood Longitude'], bar_cpy['Neighborhood'], bar_cpy['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
map_clusters.save("map_clusters.png")

### CLUSTERING  

In [64]:
#Cluster 0
cluster_0 = bar_cpy.loc[bar_cpy["Cluster Labels"]==0]
cluster_0

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arverne,0.0,0,40.589144,-73.791992,Rockaway Beach - 60th Street,40.589328,-73.789144,Beach
40,Jamaica Hills,0.0,0,40.711460,-73.796465,"MTA NYCT Q1, Q2, Q3, Q36, Q76, Q77 NICEBus N1,...",40.709230,-73.797299,Bus Station
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Taco Bell,40.715374,-73.793872,Fast Food Restaurant
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Pollo Popular,40.710027,-73.793734,Restaurant
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Jamaica Colosseum Mall,40.707243,-73.796147,Shopping Mall
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Peking Kitchen,40.710610,-73.792701,Chinese Restaurant
40,Jamaica Hills,0.0,0,40.711460,-73.796465,New Dainty Food,40.711119,-73.791925,Convenience Store
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Petland Discount Hillside Ave,40.714113,-73.792608,Pet Store
40,Jamaica Hills,0.0,0,40.711460,-73.796465,Estates Pharmacy,40.714220,-73.792572,Pharmacy
40,Jamaica Hills,0.0,0,40.711460,-73.796465,MTA Q6 bus stop,40.707124,-73.794981,Moving Target


In [65]:
#Cluster 1
cluster_1 = bar_cpy.loc[bar_cpy["Cluster Labels"]==1]
cluster_1

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
80,Woodside,0.066667,1,40.746349,-73.901842,Jollibee,40.745986,-73.901197,Fast Food Restaurant
80,Woodside,0.066667,1,40.746349,-73.901842,Izalco,40.746060,-73.899753,Latin American Restaurant
80,Woodside,0.066667,1,40.746349,-73.901842,Thailand's Center Point,40.746531,-73.900247,Thai Restaurant
80,Woodside,0.066667,1,40.746349,-73.901842,SriPraPhai,40.746342,-73.899248,Thai Restaurant
80,Woodside,0.066667,1,40.746349,-73.901842,Han Ah Reum Asian Mart,40.744644,-73.905074,Grocery Store
80,Woodside,0.066667,1,40.746349,-73.901842,Fay Da Bakery,40.745836,-73.902115,Bakery
80,Woodside,0.066667,1,40.746349,-73.901842,Casa Del Chef Bistro,40.746174,-73.899942,American Restaurant
80,Woodside,0.066667,1,40.746349,-73.901842,The Starting Gate,40.745356,-73.905170,Bar
80,Woodside,0.066667,1,40.746349,-73.901842,Sean Og's,40.744865,-73.904420,Pub
80,Woodside,0.066667,1,40.746349,-73.901842,Cafe Con Amor,40.746039,-73.899441,American Restaurant


In [66]:
#Cluster 2
cluster_2 = bar_cpy.loc[bar_cpy["Cluster Labels"]==2]
cluster_2

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
41,Kew Gardens,0.033333,2,40.705179,-73.829819,Chase Bank,40.708527,-73.830468,Bank
41,Kew Gardens,0.033333,2,40.705179,-73.829819,Beverly Hills Hair Stylist,40.708757,-73.830335,Cosmetics Shop
22,Elmhurst,0.033333,2,40.744049,-73.881656,iCook Hotpot,40.743192,-73.883348,Hotpot Restaurant
22,Elmhurst,0.033333,2,40.744049,-73.881656,JoJu,40.740689,-73.879389,Vietnamese Restaurant
22,Elmhurst,0.033333,2,40.744049,-73.881656,Terraza 7,40.747514,-73.882117,Bar
22,Elmhurst,0.033333,2,40.744049,-73.881656,Lao Bei Fang,40.741078,-73.879996,Chinese Restaurant
22,Elmhurst,0.033333,2,40.744049,-73.881656,Dek Sen,40.741753,-73.878696,Thai Restaurant
22,Elmhurst,0.033333,2,40.744049,-73.881656,Alex's Barbershop,40.741551,-73.879602,Salon / Barbershop
22,Elmhurst,0.033333,2,40.744049,-73.881656,Chao Thai ร้านชาวไทย,40.741416,-73.880102,Thai Restaurant
41,Kew Gardens,0.033333,2,40.705179,-73.829819,Kew Gardens Fish Market,40.709045,-73.830327,Fish Market


In [67]:
#Cluster 3
cluster_3 = bar_cpy.loc[bar_cpy["Cluster Labels"]==3]
cluster_3

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
56,Pomonok,0.111111,3,40.734936,-73.804861,Q65 Bus Stop (164 St & Jewel Ave),40.731919,-73.805115,Bus Station
56,Pomonok,0.111111,3,40.734936,-73.804861,Napoli's Pizzeria,40.732992,-73.804815,Pizza Place
56,Pomonok,0.111111,3,40.734936,-73.804861,JIB Lanes,40.733943,-73.810412,Bowling Alley
56,Pomonok,0.111111,3,40.734936,-73.804861,Key Food,40.733221,-73.804533,Supermarket
56,Pomonok,0.111111,3,40.734936,-73.804861,Futigi Japanese Cuisine,40.733060,-73.804720,Japanese Restaurant
56,Pomonok,0.111111,3,40.734936,-73.804861,Electric Playground,40.735492,-73.807016,Playground
56,Pomonok,0.111111,3,40.734936,-73.804861,Keyfood,40.735771,-73.801472,Grocery Store
56,Pomonok,0.111111,3,40.734936,-73.804861,Fillmore's Tavern,40.736552,-73.802801,Bar
56,Pomonok,0.111111,3,40.734936,-73.804861,Horace Harding Overpass,40.738474,-73.804718,Scenic Lookout


In [68]:
#Cluster 4
cluster_4 = bar_cpy.loc[bar_cpy["Cluster Labels"]==4]
cluster_4

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
60,Ravenswood,0.043478,4,40.761705,-73.931575,Bravo Supermarkets,40.761190,-73.931096,Grocery Store
60,Ravenswood,0.043478,4,40.761705,-73.931575,La Sala,40.763194,-73.928571,Lounge
66,Rockaway Park,0.043478,4,40.580343,-73.841534,Pickles & Pies Food Market & Deli,40.579915,-73.837182,Deli / Bodega
66,Rockaway Park,0.043478,4,40.580343,-73.841534,The Wharf,40.582680,-73.838440,Seafood Restaurant
66,Rockaway Park,0.043478,4,40.580343,-73.841534,Ciro's Italian Restaurant,40.579271,-73.836954,Pizza Place
66,Rockaway Park,0.043478,4,40.580343,-73.841534,SUBWAY,40.581786,-73.837039,Sandwich Place
66,Rockaway Park,0.043478,4,40.580343,-73.841534,Capital One Bank,40.579547,-73.837391,Bank
58,Queensboro Hill,0.041667,4,40.744572,-73.825809,FruiTea (FKA Red Apple),40.746259,-73.826011,Frozen Yogurt Shop
58,Queensboro Hill,0.041667,4,40.744572,-73.825809,Recovery Room sports bar,40.746138,-73.831099,Bar
58,Queensboro Hill,0.041667,4,40.744572,-73.825809,Hampton Jitney - Queens Airport Connection Wes...,40.742782,-73.825920,Bus Stop


In [69]:
#Cluster 5
cluster_5 = bar_cpy.loc[bar_cpy["Cluster Labels"]==5]
cluster_5

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,The Alcove,40.745929,-73.915263,American Restaurant
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Skillman Famous Pizza and Coffee Shop,40.746498,-73.917801,Pizza Place
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Red Wing,40.743350,-73.917676,Shoe Store
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Tito Rad's Grill & Restaurant,40.742477,-73.915584,Filipino Restaurant
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Sotto Le Stelle,40.743424,-73.920165,Pizza Place
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,The Lowery Bar & Kitchen,40.745138,-73.921036,Gastropub
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Sik Gaek,40.743322,-73.915336,Korean Restaurant
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Oki Japanese Restaurant,40.743312,-73.919041,Japanese Restaurant
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Rite Aid,40.744057,-73.914465,Pharmacy
76,Sunnyside Gardens,0.1,5,40.745652,-73.918193,Firefly,40.744924,-73.918830,Bistro


In [70]:
#Cluster_6
cluster_6 = bar_cpy.loc[bar_cpy["Cluster Labels"]==6]
cluster_6

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,Blissville,0.055556,6,40.737251,-73.932442,Westside Skate & Stick,40.741531,-73.933694,Sporting Goods Shop
11,Blissville,0.055556,6,40.737251,-73.932442,Tropical Restaurant,40.737414,-73.929609,Restaurant
11,Blissville,0.055556,6,40.737251,-73.932442,Red Roof Inn Queens,40.736662,-73.928114,Hotel
11,Blissville,0.055556,6,40.737251,-73.932442,MTA - B24 Bus Stop (Van Dam),40.735161,-73.937458,Bus Station
11,Blissville,0.055556,6,40.737251,-73.932442,City Cafe,40.740423,-73.934455,Chinese Restaurant
11,Blissville,0.055556,6,40.737251,-73.932442,Van Dam Deli,40.736020,-73.936580,Deli / Bodega
11,Blissville,0.055556,6,40.737251,-73.932442,Aponte Deli Grocery,40.737386,-73.929565,Deli / Bodega
11,Blissville,0.055556,6,40.737251,-73.932442,Housing Works - Warehouse,40.739020,-73.930969,Antique Shop
11,Blissville,0.055556,6,40.737251,-73.932442,Bantry Bay Publick House,40.735669,-73.934780,Bar
11,Blissville,0.055556,6,40.737251,-73.932442,Silvercup East -Stage G,40.737177,-73.935170,Movie Theater


In [71]:
#Cluster 7
cluster_7 = bar_cpy.loc[bar_cpy["Cluster Labels"]==7]
cluster_7

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Halal Truck @ Walgreens.,40.671872,-73.813426,Food Truck
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Biordi's Italian-American Deli,40.668712,-73.811585,Deli / Bodega
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Dunkin',40.667459,-73.810565,Donut Shop
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Hilton Garden Inn,40.665761,-73.806260,Hotel
71,South Ozone Park,0.076923,7,40.66855,-73.809865,SUBWAY,40.667304,-73.806493,Sandwich Place
71,South Ozone Park,0.076923,7,40.66855,-73.809865,New Times Mini Market & Deli Inc.,40.667716,-73.807998,Deli / Bodega
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Burger King,40.671371,-73.810611,Fast Food Restaurant
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Pals Oval Park,40.668634,-73.805878,Park
71,South Ozone Park,0.076923,7,40.66855,-73.809865,McDonald's,40.671602,-73.806619,Fast Food Restaurant
71,South Ozone Park,0.076923,7,40.66855,-73.809865,Back Street Park,40.666542,-73.806407,Park


### Observations and Results

Cluster	| No. of Bars  
--- | --- |
0 	 | 0  
1 	 | 6  
2 	 | 7  
3    | 1  
4 	 | 3  
5 	 | 3  
6 	 | 1  
7 	 | 1  

